In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Embedding, Lambda, Dense
from tensorflow.keras import Sequential
import keras.backend as K
from tensorflow.keras.utils import to_categorical

In [2]:
file = open('CBOW.txt', 'r')
data = file.read()
data

'The speed of transmission is an important point of difference between the two viruses. Influenza has a shorter median incubation period (the time from infection to appearance of symptoms) and a shorter serial interval (the time between successive cases) than COVID-19 virus. The serial interval for COVID-19 virus is estimated to be 5-6 days, while for influenza virus, the serial interval is 3 days. This means that influenza can spread faster than COVID-19. \n\nFurther, transmission in the first 3-5 days of illness, or potentially pre-symptomatic transmission –transmission of the virus before the appearance of symptoms – is a major driver of transmission for influenza. In contrast, while we are learning that there are people who can shed COVID-19 virus 24-48 hours prior to symptom onset, at present, this does not appear to be a major driver of transmission. \n\nThe reproductive number – the number of secondary infections generated from one infected individual – is understood to be betwe

In [3]:
words = data.split()
words

['The',
 'speed',
 'of',
 'transmission',
 'is',
 'an',
 'important',
 'point',
 'of',
 'difference',
 'between',
 'the',
 'two',
 'viruses.',
 'Influenza',
 'has',
 'a',
 'shorter',
 'median',
 'incubation',
 'period',
 '(the',
 'time',
 'from',
 'infection',
 'to',
 'appearance',
 'of',
 'symptoms)',
 'and',
 'a',
 'shorter',
 'serial',
 'interval',
 '(the',
 'time',
 'between',
 'successive',
 'cases)',
 'than',
 'COVID-19',
 'virus.',
 'The',
 'serial',
 'interval',
 'for',
 'COVID-19',
 'virus',
 'is',
 'estimated',
 'to',
 'be',
 '5-6',
 'days,',
 'while',
 'for',
 'influenza',
 'virus,',
 'the',
 'serial',
 'interval',
 'is',
 '3',
 'days.',
 'This',
 'means',
 'that',
 'influenza',
 'can',
 'spread',
 'faster',
 'than',
 'COVID-19.',
 'Further,',
 'transmission',
 'in',
 'the',
 'first',
 '3-5',
 'days',
 'of',
 'illness,',
 'or',
 'potentially',
 'pre-symptomatic',
 'transmission',
 '–transmission',
 'of',
 'the',
 'virus',
 'before',
 'the',
 'appearance',
 'of',
 'symptoms',

In [4]:
words = [w.lower().replace(".","").replace("!","").replace(",","").replace("\n","").replace("'","") for w in words]
words

['the',
 'speed',
 'of',
 'transmission',
 'is',
 'an',
 'important',
 'point',
 'of',
 'difference',
 'between',
 'the',
 'two',
 'viruses',
 'influenza',
 'has',
 'a',
 'shorter',
 'median',
 'incubation',
 'period',
 '(the',
 'time',
 'from',
 'infection',
 'to',
 'appearance',
 'of',
 'symptoms)',
 'and',
 'a',
 'shorter',
 'serial',
 'interval',
 '(the',
 'time',
 'between',
 'successive',
 'cases)',
 'than',
 'covid-19',
 'virus',
 'the',
 'serial',
 'interval',
 'for',
 'covid-19',
 'virus',
 'is',
 'estimated',
 'to',
 'be',
 '5-6',
 'days',
 'while',
 'for',
 'influenza',
 'virus',
 'the',
 'serial',
 'interval',
 'is',
 '3',
 'days',
 'this',
 'means',
 'that',
 'influenza',
 'can',
 'spread',
 'faster',
 'than',
 'covid-19',
 'further',
 'transmission',
 'in',
 'the',
 'first',
 '3-5',
 'days',
 'of',
 'illness',
 'or',
 'potentially',
 'pre-symptomatic',
 'transmission',
 '–transmission',
 'of',
 'the',
 'virus',
 'before',
 'the',
 'appearance',
 'of',
 'symptoms',
 '–',
 

In [5]:
vocab = set(words)
vocab_size=len(words)

In [6]:
words_to_idx = {word:i for i, word in enumerate(vocab)}
idx_to_words = {i:word for i, word in enumerate(vocab)}

In [7]:
words_to_idx

{'contrast': 0,
 'serial': 1,
 'interval': 2,
 'successive': 3,
 'and': 4,
 'prior': 5,
 'covid-19': 6,
 'a': 7,
 'higher': 8,
 'time': 9,
 'infections': 10,
 'or': 11,
 '(the': 12,
 'can': 13,
 'hours': 14,
 'people': 15,
 'difference': 16,
 'first': 17,
 'individual': 18,
 'in': 19,
 'infected': 20,
 'point': 21,
 'further': 22,
 'spread': 23,
 'important': 24,
 'the': 25,
 'speed': 26,
 'for': 27,
 'before': 28,
 'who': 29,
 'symptom': 30,
 '–transmission': 31,
 'period': 32,
 'than': 33,
 'context': 34,
 '3-5': 35,
 'influenza': 36,
 'driver': 37,
 'viruses': 38,
 'estimated': 39,
 'number': 40,
 'more': 41,
 'understood': 42,
 'virus': 43,
 'this': 44,
 '25': 45,
 'very': 46,
 'means': 47,
 'time-specific': 48,
 'at': 49,
 'to': 50,
 'secondary': 51,
 'an': 52,
 'symptoms)': 53,
 'present': 54,
 'reproductive': 55,
 'from': 56,
 'cases)': 57,
 'however': 58,
 'symptoms': 59,
 'major': 60,
 'generated': 61,
 '24-48': 62,
 'onset': 63,
 '2': 64,
 'difficult': 65,
 'making': 66,
 'is

In [8]:
idx_to_words

{0: 'contrast',
 1: 'serial',
 2: 'interval',
 3: 'successive',
 4: 'and',
 5: 'prior',
 6: 'covid-19',
 7: 'a',
 8: 'higher',
 9: 'time',
 10: 'infections',
 11: 'or',
 12: '(the',
 13: 'can',
 14: 'hours',
 15: 'people',
 16: 'difference',
 17: 'first',
 18: 'individual',
 19: 'in',
 20: 'infected',
 21: 'point',
 22: 'further',
 23: 'spread',
 24: 'important',
 25: 'the',
 26: 'speed',
 27: 'for',
 28: 'before',
 29: 'who',
 30: 'symptom',
 31: '–transmission',
 32: 'period',
 33: 'than',
 34: 'context',
 35: '3-5',
 36: 'influenza',
 37: 'driver',
 38: 'viruses',
 39: 'estimated',
 40: 'number',
 41: 'more',
 42: 'understood',
 43: 'virus',
 44: 'this',
 45: '25',
 46: 'very',
 47: 'means',
 48: 'time-specific',
 49: 'at',
 50: 'to',
 51: 'secondary',
 52: 'an',
 53: 'symptoms)',
 54: 'present',
 55: 'reproductive',
 56: 'from',
 57: 'cases)',
 58: 'however',
 59: 'symptoms',
 60: 'major',
 61: 'generated',
 62: '24-48',
 63: 'onset',
 64: '2',
 65: 'difficult',
 66: 'making',
 67:

In [9]:
c=2
dim=100

x_train = []
y_train = []

for index, i in enumerate(words):

    if(index>=len(words)-2) or (index<=1):
        continue
        
    start = index - c
    end = index + c + 1
    context=words[start:end]
    context.remove(i)
    target = i
    
    x_train.append([words_to_idx[w] for w in context])
    y_train.append([words_to_idx[target]])

In [10]:
x_train = np.array(x_train)
y_train = np.array(y_train)
y_train = to_categorical(y_train, vocab_size)
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [11]:
x_train

array([[ 25,  26,  92,  67],
       [ 26, 100,  67,  52],
       [100,  92,  52,  24],
       [ 92,  67,  24,  21],
       [ 67,  52,  21, 100],
       [ 52,  24, 100,  16],
       [ 24,  21,  16,  89],
       [ 21, 100,  89,  25],
       [100,  16,  25,  80],
       [ 16,  89,  80,  38],
       [ 89,  25,  38,  36],
       [ 25,  80,  36,  94],
       [ 80,  38,  94,   7],
       [ 38,  36,   7,  96],
       [ 36,  94,  96,  71],
       [ 94,   7,  71, 101],
       [  7,  96, 101,  32],
       [ 96,  71,  32,  12],
       [ 71, 101,  12,   9],
       [101,  32,   9,  56],
       [ 32,  12,  56,  98],
       [ 12,   9,  98,  50],
       [  9,  56,  50,  86],
       [ 56,  98,  86, 100],
       [ 98,  50, 100,  53],
       [ 50,  86,  53,   4],
       [ 86, 100,   4,   7],
       [100,  53,   7,  96],
       [ 53,   4,  96,   1],
       [  4,   7,   1,   2],
       [  7,  96,   2,  12],
       [ 96,   1,  12,   9],
       [  1,   2,   9,  89],
       [  2,  12,  89,   3],
       [ 12,  

In [12]:
cbow = Sequential()
cbow.add(Embedding(input_dim=vocab_size, output_dim=dim, input_length=c*2))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(dim,)))
cbow.add(Dense(vocab_size, activation="softmax"))
cbow.compile(loss="categorical_crossentropy", optimizer=SGD(), metrics=['accuracy'])
cbow.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 100)            18600     
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 186)               18786     
                                                                 
Total params: 37386 (146.04 KB)
Trainable params: 37386 (146.04 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
h = cbow.fit(x_train, y_train, epochs=5000)

Epoch 1/5000
6/6 [==============================] - 0s 2ms/step - loss: 5.1437 - accuracy: 0.0495
Epoch 2/5000
6/6 [==============================] - 0s 2ms/step - loss: 5.1429 - accuracy: 0.0495
Epoch 3/5000
6/6 [==============================] - 0s 2ms/step - loss: 5.1422 - accuracy: 0.0495
Epoch 4/5000
6/6 [==============================] - 0s 2ms/step - loss: 5.1414 - accuracy: 0.0495
Epoch 5/5000
6/6 [==============================] - 0s 2ms/step - loss: 5.1406 - accuracy: 0.0495
Epoch 6/5000
6/6 [==============================] - 0s 3ms/step - loss: 5.1398 - accuracy: 0.0495
Epoch 7/5000
6/6 [==============================] - 0s 2ms/step - loss: 5.1391 - accuracy: 0.0495
Epoch 8/5000
6/6 [==============================] - 0s 2ms/step - loss: 5.1383 - accuracy: 0.0495
Epoch 9/5000
6/6 [==============================] - 0s 2ms/step - loss: 5.1375 - accuracy: 0.0495
Epoch 10/5000
6/6 [==============================] - 0s 3ms/step - loss: 5.1367 - accuracy: 0.0495
Epoch 11/5000
6/6 [

6/6 [==============================] - 0s 2ms/step - loss: 5.0814 - accuracy: 0.0495
Epoch 84/5000
6/6 [==============================] - 0s 2ms/step - loss: 5.0806 - accuracy: 0.0495
Epoch 85/5000
6/6 [==============================] - 0s 2ms/step - loss: 5.0799 - accuracy: 0.0495
Epoch 86/5000
6/6 [==============================] - 0s 2ms/step - loss: 5.0792 - accuracy: 0.0495
Epoch 87/5000
6/6 [==============================] - 0s 2ms/step - loss: 5.0784 - accuracy: 0.0495
Epoch 88/5000
6/6 [==============================] - 0s 2ms/step - loss: 5.0777 - accuracy: 0.0495
Epoch 89/5000
6/6 [==============================] - 0s 2ms/step - loss: 5.0770 - accuracy: 0.0495
Epoch 90/5000
6/6 [==============================] - 0s 2ms/step - loss: 5.0762 - accuracy: 0.0495
Epoch 91/5000
6/6 [==============================] - 0s 2ms/step - loss: 5.0755 - accuracy: 0.0495
Epoch 92/5000
6/6 [==============================] - 0s 2ms/step - loss: 5.0748 - accuracy: 0.0495
Epoch 93/5000
6/6 [=====

6/6 [==============================] - 0s 2ms/step - loss: 5.0226 - accuracy: 0.0495
Epoch 166/5000
6/6 [==============================] - 0s 2ms/step - loss: 5.0219 - accuracy: 0.0495
Epoch 167/5000
6/6 [==============================] - 0s 2ms/step - loss: 5.0212 - accuracy: 0.0495
Epoch 168/5000
6/6 [==============================] - 0s 2ms/step - loss: 5.0205 - accuracy: 0.0495
Epoch 169/5000
6/6 [==============================] - 0s 2ms/step - loss: 5.0198 - accuracy: 0.0495
Epoch 170/5000
6/6 [==============================] - 0s 2ms/step - loss: 5.0192 - accuracy: 0.0495
Epoch 171/5000
6/6 [==============================] - 0s 2ms/step - loss: 5.0185 - accuracy: 0.0495
Epoch 172/5000
6/6 [==============================] - 0s 2ms/step - loss: 5.0178 - accuracy: 0.0495
Epoch 173/5000
6/6 [==============================] - 0s 2ms/step - loss: 5.0171 - accuracy: 0.0495
Epoch 174/5000
6/6 [==============================] - 0s 2ms/step - loss: 5.0164 - accuracy: 0.0495
Epoch 175/5000


6/6 [==============================] - 0s 2ms/step - loss: 4.9675 - accuracy: 0.0495
Epoch 248/5000
6/6 [==============================] - 0s 2ms/step - loss: 4.9668 - accuracy: 0.0495
Epoch 249/5000
6/6 [==============================] - 0s 2ms/step - loss: 4.9662 - accuracy: 0.0495
Epoch 250/5000
6/6 [==============================] - 0s 2ms/step - loss: 4.9655 - accuracy: 0.0495
Epoch 251/5000
6/6 [==============================] - 0s 2ms/step - loss: 4.9649 - accuracy: 0.0495
Epoch 252/5000
6/6 [==============================] - 0s 2ms/step - loss: 4.9642 - accuracy: 0.0495
Epoch 253/5000
6/6 [==============================] - 0s 2ms/step - loss: 4.9636 - accuracy: 0.0495
Epoch 254/5000
6/6 [==============================] - 0s 2ms/step - loss: 4.9629 - accuracy: 0.0495
Epoch 255/5000
6/6 [==============================] - 0s 2ms/step - loss: 4.9623 - accuracy: 0.0495
Epoch 256/5000
6/6 [==============================] - 0s 2ms/step - loss: 4.9616 - accuracy: 0.0495
Epoch 257/5000


6/6 [==============================] - 0s 2ms/step - loss: 4.9159 - accuracy: 0.0495
Epoch 330/5000
6/6 [==============================] - 0s 2ms/step - loss: 4.9153 - accuracy: 0.0495
Epoch 331/5000
6/6 [==============================] - 0s 2ms/step - loss: 4.9147 - accuracy: 0.0495
Epoch 332/5000
6/6 [==============================] - 0s 2ms/step - loss: 4.9141 - accuracy: 0.0495
Epoch 333/5000
6/6 [==============================] - 0s 2ms/step - loss: 4.9135 - accuracy: 0.0495
Epoch 334/5000
6/6 [==============================] - 0s 2ms/step - loss: 4.9129 - accuracy: 0.0495
Epoch 335/5000
6/6 [==============================] - 0s 2ms/step - loss: 4.9123 - accuracy: 0.0495
Epoch 336/5000
6/6 [==============================] - 0s 2ms/step - loss: 4.9116 - accuracy: 0.0495
Epoch 337/5000
6/6 [==============================] - 0s 2ms/step - loss: 4.9110 - accuracy: 0.0495
Epoch 338/5000
6/6 [==============================] - 0s 2ms/step - loss: 4.9104 - accuracy: 0.0495
Epoch 339/5000


...

6/6 [==============================] - 0s 2ms/step - loss: 2.7876 - accuracy: 0.4231
Epoch 4631/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.7869 - accuracy: 0.4231
Epoch 4632/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.7862 - accuracy: 0.4231
Epoch 4633/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.7855 - accuracy: 0.4231
Epoch 4634/5000
6/6 [==============================] - 0s 3ms/step - loss: 2.7849 - accuracy: 0.4231
Epoch 4635/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.7842 - accuracy: 0.4231
Epoch 4636/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.7836 - accuracy: 0.4231
Epoch 4637/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.7829 - accuracy: 0.4231
Epoch 4638/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.7822 - accuracy: 0.4231
Epoch 4639/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.7815 - accuracy: 0.4231
Epoch 

6/6 [==============================] - 0s 2ms/step - loss: 2.7334 - accuracy: 0.4286
Epoch 4712/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.7327 - accuracy: 0.4286
Epoch 4713/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.7321 - accuracy: 0.4286
Epoch 4714/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.7314 - accuracy: 0.4286
Epoch 4715/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.7307 - accuracy: 0.4286
Epoch 4716/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.7300 - accuracy: 0.4286
Epoch 4717/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.7294 - accuracy: 0.4286
Epoch 4718/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.7287 - accuracy: 0.4286
Epoch 4719/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.7280 - accuracy: 0.4286
Epoch 4720/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.7274 - accuracy: 0.4286
Epoch 

6/6 [==============================] - 0s 2ms/step - loss: 2.6792 - accuracy: 0.4286
Epoch 4793/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.6785 - accuracy: 0.4286
Epoch 4794/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.6779 - accuracy: 0.4286
Epoch 4795/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.6772 - accuracy: 0.4286
Epoch 4796/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.6766 - accuracy: 0.4286
Epoch 4797/5000
6/6 [==============================] - 0s 4ms/step - loss: 2.6759 - accuracy: 0.4286
Epoch 4798/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.6752 - accuracy: 0.4286
Epoch 4799/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.6745 - accuracy: 0.4286
Epoch 4800/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.6739 - accuracy: 0.4286
Epoch 4801/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.6732 - accuracy: 0.4286
Epoch 

6/6 [==============================] - 0s 2ms/step - loss: 2.6251 - accuracy: 0.4505
Epoch 4874/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.6245 - accuracy: 0.4505
Epoch 4875/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.6238 - accuracy: 0.4505
Epoch 4876/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.6231 - accuracy: 0.4505
Epoch 4877/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.6225 - accuracy: 0.4505
Epoch 4878/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.6218 - accuracy: 0.4505
Epoch 4879/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.6211 - accuracy: 0.4505
Epoch 4880/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.6204 - accuracy: 0.4505
Epoch 4881/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.6198 - accuracy: 0.4505
Epoch 4882/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.6191 - accuracy: 0.4505
Epoch 

6/6 [==============================] - 0s 2ms/step - loss: 2.5711 - accuracy: 0.4615
Epoch 4955/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.5704 - accuracy: 0.4615
Epoch 4956/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.5697 - accuracy: 0.4615
Epoch 4957/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.5690 - accuracy: 0.4615
Epoch 4958/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.5684 - accuracy: 0.4615
Epoch 4959/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.5677 - accuracy: 0.4615
Epoch 4960/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.5670 - accuracy: 0.4615
Epoch 4961/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.5664 - accuracy: 0.4615
Epoch 4962/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.5657 - accuracy: 0.4615
Epoch 4963/5000
6/6 [==============================] - 0s 2ms/step - loss: 2.5650 - accuracy: 0.4615
Epoch 

In [18]:
pred = cbow.predict(x_train)

6/6 [==============================] - 0s 1ms/step


In [19]:
pred = pred.argmax(axis=1)
pred

array([100,  92,  25, 100,  92,  92, 100,  92,  92,  25, 100,  36,  36,
         4,   7,  96,  96,   7,  12,  12,   9,  84,  84,  50,  67, 100,
        37,  60,   7,  96,   1,   2,  25,   9,  89,  89,  43,  27,   6,
        43,  25,   1,  43,  25,   6,  43,  67,  50,  50,  84,  84,  50,
        36,  27,   6,  43,  25,   1,   2,  67,  50,  50,  44,  85,  85,
        36,  85,   6,  43,  33,   6,  27,  43, 100,  25, 100,  92,  25,
       100,  92,  92, 100, 100,  92,  92, 100,  25,  43, 100,  25,  92,
       100,  79,  60,  67,   7,  60,  37, 100,  92,  25,  36,  36,  27,
        36,  36,  85,  85,  97,  85,  85,  36,  36,  43,  27,   6,  43,
        27,  67,  84,  50,  84,  84,  50,  50,  50,  50,  84,  50,  50,
        84,   7,  60,  37, 100,  92,  25, 100,  40,  79,  25,  40, 100,
        25,  50,  50,  56,  50,  67,  67,  79,  67,  50,  50,  84,  84,
        50,  36,  36,  27,   6,  43,   6,   6,  27,  36,  36,  27,  27,
        36,  36,  27,  36,  85,  85,  36,  36,   4,   4, 100,   

In [20]:
i = 0
for context in x_train:
    print("==================Context================")
    for w in context:
        print(idx_to_words[w])
    print("=================Predicted================")
    print(idx_to_words[pred[i]])
    i+=1

==================Context================
the
speed
transmission
is
=================Predicted================
of
==================Context================
speed
of
is
an
=================Predicted================
transmission
==================Context================
of
transmission
an
important
=================Predicted================
the
==================Context================
transmission
is
important
point
=================Predicted================
of
==================Context================
is
an
point
of
=================Predicted================
transmission
==================Context================
an
important
of
difference
=================Predicted================
transmission
==================Context================
important
point
difference
between
=================Predicted================
of
==================Context================
point
of
between
the
=================Predicted================
transmission
==================Context================
of
difference